<a href="https://www.kaggle.com/code/advaitchavan/sem-6-mini-project-book-recommendation-system?scriptVersionId=127862199" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1>Book Recommendation Engine using the Collaborative Filtering(CF) Algorithm</h1> 

<h4>A recommendation system is a subclass of facts filtering systems that seeks to expect the rating or the choice a user would possibly deliver to an item. In simple phrases, it's miles an set of rules that shows applicable objects to customers.</h4>
<h4>One of the effective personalization technologies powering the adaptive web is collaborative filtering. Collaborative filtering (CF) is the manner of filtering or comparing objects via the critiques of different people. CF technology brings collectively the reviews of massive interconnected communities at the web, assisting filtering of widespread portions of statistics. in this project we introduce the middle concepts of collaborative filtering, its primary makes use of for users of the adaptive internet, the theory and practice of CF algorithms, and layout decisions regarding rating systems and acquisition of rankings.</h4>

In [1]:
import pandas as pd
import numpy as np
from joblib import dump

In [2]:
ratings_data = pd.read_csv("../input/book-crossing-dataset/BX-Book-Ratings.csv",sep=";",error_bad_lines=False,encoding="latin-1")
ratings_data_copy = ratings_data.copy()

users_data = pd.read_csv("../input/book-crossing-dataset/BX-Users.csv",encoding="latin-1",sep=";",error_bad_lines=False)
users_data_copy = users_data.copy()

books_data = pd.read_csv("../input/book-crossing-dataset/BX-Books.csv",sep=";",encoding="latin-1",error_bad_lines=False,engine="python")
books_data_copy = books_data.copy()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 815: ';' expected after '"'
Skipping line 1679: ';' expected after '"'
Skipping line 2531: ';' expected after '"'
Skipping line 2640: ';' expected after '"'
Skipping line 3236: ';' expected after '"'
Skipping line 3256: ';' expected after '"'
Skipping line 4906: ';' expected after '"'
Skipping line 6452: ';' expected after '"'
Skipping line 8349: ';' expected after '"'
Skipping line 9669: ';' expected after '"'
Skipping line 9723: ';' expected after '"'
Skipping line 10010: ';' expected after '"'
Skipping line 10192: ';' expected after '"'
Skipping line 10754: ';' expected after '"'
Skipping line 10824: ';' expected after '"'
Skipping line 10862: ';' expected after '"'
Skipping line 11442: ';' expected after '"'
Skipping line 11584: ';' ex

<h5> Keeping the Required Columns that will be required for Our ML Training </h5>

In [3]:
books_data = books_data[["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher"]]

<h5>Changing the column names for ease.</h5>

In [4]:
books_data.rename(columns={"Book-Title":"title","Book-Author":"author","Year-Of-Publication":"year","Publisher":"publisher"},inplace=True)
users_data.rename(columns={"User-ID":"user_id","Location":"location","Age":"age"},inplace=True)
ratings_data.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

<h5>Finding out those users who have rated the books</h5>

In [5]:
ratings_data["user_id"].value_counts().shape

(105283,)

**105283 users have actually rated the books**

**Selecting only those users who have given more than 180 ratings, so as to  to improve our accuracy**

In [6]:
a = ratings_data["user_id"].value_counts()>=180
b = a[a].index

**Selecting only those rows of the user id who have given us more than 180 ratings**

In [7]:
ratings_data = ratings_data[ratings_data["user_id"].isin(b)]

**Merging rating_data and users_data based on ISBN column**

In [8]:
rated_books_data = ratings_data.merge(books_data,on="ISBN")

**Calculate the number of books that have been rated many times**

In [9]:
no_of_ratings_data = rated_books_data.groupby("title")["rating"].count().reset_index()
no_of_ratings_data.rename(columns={"rating":"number_of_rating"},inplace=True)
no_of_ratings_data

,title,number_of_rating
0,A Light in the Storm: The Civil War Diary of ...,3
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
163981,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
163982,Ã?Â?lpiraten.,1
163983,Ã?Â?rger mit Produkt X. Roman.,1
163984,Ã?Â?stlich der Berge.,1


**Merging no_of_ratings_data with rated_books_data**

In [10]:
final_rating_data = rated_books_data.merge(no_of_ratings_data,on="title")

In [11]:
final_rating_data

,user_id,ISBN,rating,title,author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
...,...,...,...,...,...,...,...,...
507749,276680,089587167X,7,Something Blue,Jean Christopher Spaugh,1997,John F. Blair Publisher,1
507750,276680,1402201435,4,The One True Ocean,Sarah Beth Martin,2003,Sourcebooks Landmark,1
507751,276680,1564407284,0,"The Old Sturbridge Village Cookbook, 2nd : Aut...",Caroline Sloat,1995,Globe Pequot,1
507752,276680,1582343594,0,City of Masks : A Cree Black Novel,Daniel Hecht,2004,Bloomsbury USA,1


**Selecting books with 50 or more number of rating**

In [12]:
final_rating_data = final_rating_data[final_rating_data["number_of_rating"]>=50]

In [13]:
final_rating_data

,user_id,ISBN,rating,title,author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,83
...,...,...,...,...,...,...,...,...
278443,258185,0515118656,0,The Cove,Catherine Coulter,1996,Jove Books,50
278444,261829,0515118656,7,The Cove,Catherine Coulter,1996,Jove Books,50
278445,264082,0515118656,0,The Cove,Catherine Coulter,1996,Jove Books,50
278446,264321,0515118656,0,The Cove,Catherine Coulter,1996,Jove Books,50


**drop any duplicates from final_rating_data**

In [14]:
final_rating_data.drop_duplicates(["user_id","title"],inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [15]:
final_rating_data.shape

(66083, 8)

**Creating pivot table with col=user,index=books,values=rating**

In [16]:
rating_pivot_table = final_rating_data.pivot_table(columns="user_id",index="title",values="rating")
rating_pivot_table.fillna(0,inplace=True)

**We will cluster, so the 0 wil be less important for our algorigthm so we will convert this pivot table to sparse matrix and the sparse matrix will be used for building model**

In [17]:
from scipy.sparse import csr_matrix
sparse_matrix = csr_matrix(rating_pivot_table)

<h2> MODEL BUILDING </h2>

In [18]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [19]:
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute')

**calculating distance of each book with every other books**

In [20]:
distances,suggestions=model.kneighbors(rating_pivot_table.iloc[0,:].values.reshape(1,-1),n_neighbors=6)

In [21]:
suggestions

array([[  0, 401,   8, 353,  35, 354]])

In [22]:
def recommend_books(book_name):
  book_index = np.where(rating_pivot_table.index==book_name)[0][0]
  distances , suggestions = model.kneighbors(rating_pivot_table.iloc[book_index,:].values.reshape(1,-1),n_neighbors=6)
  suggestions = np.ravel(suggestions, order='C') #2d to 1d array
  for i in suggestions:
    print(rating_pivot_table.index[i])

In [23]:
recommend_books("1984")

1984
No Safe Place
A Civil Action
Malice
Abduction
Master of the Game


In [24]:
recommend_books("Animal Farm")

Animal Farm
Exclusive
Jacob Have I Loved
Second Nature
The Playboy
Personal Injuries


**Here our recommendation model is complete**

**Making book image dataframe**

In [25]:
books_image_data = books_data_copy[["Book-Title","Image-URL-M"]]
books_image_data.rename(columns={"Book-Title":"title","Image-URL-M":"image"},inplace=True)
books_image_data = books_image_data[books_image_data["title"].isin(rating_pivot_table.index)]
books_image_data.drop_duplicates(subset=["title"],keep='first',inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
books_image_data

,title,image
5,The Kitchen God's Wife,http://images.amazon.com/images/P/0399135782.0...
18,The Testament,http://images.amazon.com/images/P/0440234743.0...
19,Beloved (Plume Contemporary Fiction),http://images.amazon.com/images/P/0452264464.0...
26,Wild Animus,http://images.amazon.com/images/P/0971880107.0...
27,Airframe,http://images.amazon.com/images/P/0345402871.0...
...,...,...
24940,Guardian Angel,http://images.amazon.com/images/P/0440213991.0...
25871,Between Friends,http://images.amazon.com/images/P/155166674X.0...
29137,M Is for Malice,http://images.amazon.com/images/P/0449223604.0...
34138,Nothing Lasts Forever,http://images.amazon.com/images/P/0688084915.0...


<h2>Pickling Our Model</h2>

In [27]:
#import pickle

In [28]:
#pickle.dump(rating_pivot_table,open("rating_table.pkl","wb"))
dump(rating_pivot_table, 'rating_table.joblib')

['rating_table.joblib']

In [29]:
#pickle.dump(books_image_data,open("books_image_data.pkl","wb"))
dump(books_image_data, 'books_image_data.joblib')

['books_image_data.joblib']

<h1> Webpage Related </h1>

In [30]:
import pandas as pd
import numpy as np
from joblib import load

In [31]:
#rating_table = pickle.load(open("./rating_table.pkl","rb"))
rating_table = load('rating_table.joblib')
#books_image_data = pickle.load(open("./books_image_data.pkl","rb"))
books_image_data = load('books_image_data.joblib')

In [32]:
from scipy.sparse import csr_matrix
new_sparse_matrix = csr_matrix(rating_table)

In [33]:
from sklearn.neighbors import NearestNeighbors
model2 = NearestNeighbors(algorithm='brute')

In [34]:
model2.fit(new_sparse_matrix)

NearestNeighbors(algorithm='brute')

<h2>Function for recommending books</h2>

In [35]:
def rec(book_name):
  recommended_books = []
  image_url = []
  book_index = np.where(rating_table.index==book_name)[0][0]
  distances , suggestions = model2.kneighbors(rating_table.iloc[book_index,:].values.reshape(1,-1),n_neighbors=6)
  suggestions = np.ravel(suggestions, order='C') #2d to 1d array
  for i in suggestions:
    recommended_books.append(rating_table.index[i])
  
  for i in recommended_books:
    image_url.append(books_image_data[books_image_data["title"] == i ].image.to_string(index=False))

    
  return recommended_books,image_url

**Function to get the images**

In [36]:
def image(book_list):
  image_url = []
  for i in book_list:
    image_url.append(books_image_data[books_image_data["title"] == i ].image.to_string(index=False))
  return image_url

In [37]:
rec("Animal Farm")[1]

['http://images.amazon.com/images/P/0451526341.01...',
 'http://images.amazon.com/images/P/0446604232.01...',
 'http://images.amazon.com/images/P/0380564998.01...',
 'http://images.amazon.com/images/P/0399139087.01...',
 'http://images.amazon.com/images/P/0446610550.01...',
 'http://images.amazon.com/images/P/0374281947.01...']

In [38]:
books_image_data[books_image_data.title=="Exclusive"]

,title,image
19105,Exclusive,http://images.amazon.com/images/P/0446604232.0...


In [39]:
import pandas as pd
import numpy as np
from joblib import load
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


rating_table = load('rating_table.joblib')
books_image_data = load('books_image_data.joblib')

books_name = rating_table.index.to_list()


sparse_matrix = csr_matrix(rating_table)
model = NearestNeighbors(algorithm='brute')
model.fit(sparse_matrix)


#Function for recommending books
def recommend(book_name):
  recommended_books = []
  image_url = []
  book_index = np.where(rating_table.index==book_name)[0][0]
  distances , suggestions = model.kneighbors(rating_table.iloc[book_index,:].values.reshape(1,-1),n_neighbors=5)
  suggestions = np.ravel(suggestions, order='C') #2d to 1d array
  for i in suggestions:
    recommended_books.append(rating_table.index[i])

  for i in recommended_books:
    image_url.append(books_image_data[books_image_data["title"] == i ].image.to_string(index=False))
    
  return recommended_books,image_url


In [40]:
recommend('Animal Farm')

(['Animal Farm',
  'Exclusive',
  'Jacob Have I Loved',
  'Second Nature',
  'The Playboy'],
 ['http://images.amazon.com/images/P/0451526341.01...',
  'http://images.amazon.com/images/P/0446604232.01...',
  'http://images.amazon.com/images/P/0380564998.01...',
  'http://images.amazon.com/images/P/0399139087.01...',
  'http://images.amazon.com/images/P/0446610550.01...'])